# Layers

ResNet: x + layer(x)
Pretty much always use this, or densenet. It can help to do larger skip connections if there is a connection between the data (for example, concat to deconv input the original input of conv of the same size , for things like UNet where it goes big then small than big again. UNet is for when the size of your output is similar to size of input and aligned with it. There is no point in doing cross connections when it's not aligned)

Instead of doing 7x7 at start, do three 3x3 convs in a row (ResNet-C from bag of tricks), c_in inputs, (c_in+1)\*8 for output channels, \*8 is because nvidia likes things to be a multiple of 8, c_in+1 so we dynamically adjust based on number of input channels (if it’s some weird image domain where we have 15 input channels than we adapt accordingly)

For a specific res block: we can still do a stride two, and then just do avgPool(2x2, stride=2) for the identity layer. For the non-identity piece, do conv 1x1, conv 3x3 with stride 2, then conv 1x1 again. Read bag of tricks paper to see their recommendations, there’s some others that do different things for different sized networks, and it’s generally about how to think through arch design - basically about thinking hard about what you are doing 

In general, look carefully at inputs and outputs and adapt to them

Usually you actually wrap ResNets around two layers in a row, such as two convs in a row 

Do a maxpool and avgPool at the end and concat them together 

DenseNet: concat(x, layer(x)) (but also nesting these, make it grow linearly and not exponentially by concat inputs from each prev layer, so kth will have all previous k inputs concat, including original input)
Memory intensive, but less parameters, so can be good for small datasets. In practice for images we do a few concats and layers, then do a deconv and start over with concats and do a few concats and layers, etc

BatchNorm: with this you can really increase the learning rate. Keeping a running average of sum and squares, since variance is just E(X^2) - E(X)^2, and use that in training and test. Also divide by (1-momentum^numItersSoFar) to “debias” which adjusts for starting with zero (quickly adapts to correct value instead of taking a little while). 

For ResNets+BatchNorm, they stack three than add input, it’s good to have the last batch norm before the add to original input to initially have a multiply bias as 0 so it starts out just doing identity then only uses it as needed. This lets you train large models at large learning rates (or giant batches and normal size learning rates)


FitOneCycle: cycle learning rates

Pre-Training on a larger dataset

Convolutions and deconvolutions

When you have trouble getting loss function to capture what you want for a generative task but you have positive examples, you can just use a GAN discriminator to learn your loss. Alternatively, you can use "feature losses" aka "perceptual losses" that just use the activations of a pretrained network on image net

Encoder and Decoder framework

LSTM/GRU are for regularization and smoothing the loss surface

Progressive resizing: train on low res images, then higher res, then highest res.

The larger the batch size, the smaller the learning rate you should have (and vice versa). If a technique says it lets them use bigger batch sizes, it also will help for same size batch and higher learning rate (and vice versa). Higher learning rate/batch size also often means you generalize better. 

Initialization:
Generally, check to see if you have mean zero std one for each layer. Practically, what you actually want is no shrinking or exploding values for weights, which would lead to shrinking or exploding values for gradients.

For RELU, multiply by 2 to fix std, subtract 0.5 to fix below (those values aren’t quite right, but they are close, todo: calculation to determine exact)

Kaiming initialization: divide by square root of values to ensure mean zero std one. When you use RELU you need to multiply by 2 (technically the exact calculations should be higher I think?).
They suggest 0 mean sqrt(2)/sqrt(number of input activations) std Gaussian distribution
init.kaiming_normal(w, mode=‘fan_out’) fan determines whether you divide by number of inputs or number of outputs 

Xavier initialization: uniform from +-sqrt(6)/sqrt(num_inputs+num_outputs)

LSUV: Instead is careful init, just try your first minibatch, then adjust biases by mean and then divide weights by std. keep doing this until mean is close to zero and std is close to 1.0 (they actually repeat changing bias until means are zero, then repeat dividing by std until var is 1.0)


Optimizer Strats:
AdamW

It’s good to ramp quickly up to high learning rate, then slowly decrease it, because you need to be at a high learning rate for quite a bit of time 

It’s important to debias (divide by (1-momentum)^stepI. Adam is just debiased momentum, momentum for grad and then divide by momentum for squared grad. With Adam eps, the smaller the eps, the more the dividing by matters, do the trick is to make eps 0.001 to 0.1 so we don’t multiply by really large values when the sum of squared is very small

LAMB: look at it, it uses the average over the layer, because you don’t want to divide by the single value cause it might be small


Activations:
Softmax is only good when you are sure every datapoint has exactly one class, no more and no less. If this is not the case, you should use binomial (exp(x)/(1+exp(x))).


Other:

Cross validation is useful if you don’t have enough data, you can tell you don’t if your variability of validation set is too high between runs, in that case you can use cross validation, but otherwise you don’t really need it. 

It can be helpful to look at the means and standard deviations of the activations (the “telemetry” of the model). You want them to be mean zero std one. You can also look at a histogram of activations over time (see 06_cuda_cnn_hooks_init for code), if you see a dense line around the bottom that’s a sign lots of the activations are near zero which is bad. See more experiments there for like seeing that 90% of your activations are mostly zero. By looking at this, we tweaked RELU to make it leaky and have an offset and that really helped.

Look at how many numbers are going in and out from each operation. It’s best to have number coming out less than number coming in. (First layer in imagenet is often 3x7x7 because it has 32 channels (outputs), but 3x3x3 would only be 27 inputs).

Look at “Bag of Tricks for image classification with Convolutional Neural Networks” for lots of tips. First layer should usually be 3x3xcolors, and output #channels = highest power of two less than 3*3*colors, they can go deeper, same receptive field, faster, etc. any time you don’t have a 3x3 kernel, think hard about if that makes sense 

Whenever you add an epsilon for numerical stability (like adding to variance before dividing in batch norm to prevent divide by zero), consider making that epsilon a learnable parameter 

Whenever you have a fixed value for all things in a layer (momentum, epsilon, etc), consider instead making it learnable parameter per output instead (or per a small bundle of outputs if that’s too many new Params)

SGDR: Stochastic Gradient Descent with Warm Restarts

replacing squared with abs often works better (for example, using it for variance for mean absolute deviation makes it less resistant to outliers). Often consider substitute functions and be willing to question standard practice until you are given a solid reason for it.


Regularization:



Data augmentation: try different downsampling strats and see which preserve the features you care about (or use all of them). Think about when you do this for best speed (often applying to bytes is faster than applying to floats, so using things like PIL transforms is good). Be careful not to do destructive transformations, and try to do them together. Think in terms of your time budget (time*total data points) when evaluating how slow they can be 

One great data augmentation strat is zooming and scaling in on random pieces. For images, perspective warping is good. For sound and text, just take small pieces. Generally, you want to use physically reasonable transformations. Some augmentation ends up cutting out the things, so that requires you do deal with noisy labels, but generally it’s good.

Pytorch can do data augmentation on the GPU using affine_grid

Look at 10_augmentation.ipynb for how to do GPU

Mixup: do randomly weighted averages of multiple pieces of data from different classes (20% dog, 80% gas station) then do loss accordingly. This also might work on things other than just the input and output labels. Sample from beta distr for determining prs, alpha=0.0 means only 0.0 or 1.0, alpha=1.0 means uniform, so pick something in the middle, you can tweak this

Label smoothing: have the model predict 0.9 for correct class and 0.1/numRemaining for others. This is a great way of handling noisy data. 

Cutout: delete a square and replace it with black, or random pixels, or a different image.



Time everything. 

Think about how things influence each other. For example, batch norm cancels out weight decay sort of, but doing both still kinda helps, and we don’t know why. 
Flooding: when train loss gets too low, do gradient ascent until it get backs to a higher train loss. Repeat doing that for a while. In practice, maybe do this whenever validation set accuracy is getting worse?

Noise injection: https://stats.stackexchange.com/a/249243/78563, it's similar to dropout (randomly add noise to each weight), but not quite the same

Weight decay/L2 weight regularization
You can also add to your loss the L2 norm of activations
For RNN they also add to your loss the L2 norm of the difference in hidden vector at each sequence step (so it penalizes changing the hidden vector a lot) - temporal regularization. 
That’s a more general principle: difference in activations between layers or other things. Try lots of variations of this

Dropout: weight/bias dropout, dropout of activations, dropout a word for embeddings (zero the entire embedding vector for a random set of words), for RNN dropout same activation consistently over a sequence, etc.
“Averaging weights leads to Wider Optima and Better Generalization”
“Decoupled Weight Decay Regularization”
“Mixup: Beyond empirical risk minimization”: does weighted averages of training samples



Scaling:
Gradient Accumulation: If you zero the gradient every k batches, you can do model updates every k iterations, effectively multiplying the batch size by k while using less memory 

Stochastic Weight Averaging (SWA) for ensemble learning

